In [ ]:
###################################################################################################
#
# algorithm_stages.py 	(c) Ian Williams, Adnan Khan, Matt McQuinn
#     				    	        ianw89@live.com
#
#  This produces Figure 1 of Khan, Williams, McQuinn 2022
#
###################################################################################################

from cgmbrush.cgmbrush import *
from cgmbrush.plots.plots import *
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

curve_thickness = 2

date = '2022-04-04'
res = 32 # for the STH 256 and 512
resolution = 32 # for the other profiles data

u = 100 # zooms into 100 of 256 pixels. (140 Mpc of the 250/h Mpc box)
p = 40 * resolution # red box / zoom-in region for profile comparison


def algorithm_stages_plot(raw_field, removed_field, addition_field, final_field, orig_grid_size, vmax, zoom_rect: bool):
    vmin = 0
    fig, axs = fields_comparison_plot( [('Raw density field', raw_field), ('Halos removed', removed_field), ('Halo-associated gas', addition_field), ('Final halo density field', final_field)], vmin, vmax)
    
    if zoom_rect:
        rect = Rectangle((0,0),p,p,linewidth=1,edgecolor='r',facecolor="none")
        axs[3].add_patch(rect)
    
    extra = ''
    if zoom_rect:
        extra = '_wbox'
    filename = 'implot_add_remove_halos_{}{}.pdf'.format(orig_grid_size, extra)
    saveFig(filename, fig, bbox_inches='tight')
    

In [ ]:
c = Configuration(SphericalTophatProfile(), resolution=res, provider=BolshoiProvider())
c.datestamp = date

original_field = c.provider.get_density_field(0, 256)
removed_field = c.get_removed_field()[0]
added_field = c.get_addition_field()[0]
final_field = c.get_final_field()[0]

c512 = Configuration(SphericalTophatProfile(), resolution=res, provider=BolshoiProvider(), den_grid_size=512)
c512.datestamp = date

original_field512 = c512.provider.get_density_field(0, 512)
removed_field512 = c512.get_removed_field()[0]
added_field512 = c512.get_addition_field()[0]
final_field512 = c512.get_final_field()[0]


In [ ]:
# get rid of data we don't need
c.clear_results() 
c512.clear_results()

In [ ]:
print("Sanity check on mean density of fields for 256")
print(np.mean(original_field))
print(np.mean(removed_field))
print(np.mean(added_field))
print(np.mean(final_field))

print("Sanity check on mean density of fields for 512")
print(np.mean(original_field512))
print(np.mean(removed_field512))
print(np.mean(added_field512))
print(np.mean(final_field512))

In [ ]:
#plt.figure(figsize=(20,20))
#print(original_field512.shape)
#print(np.mean(original_field512))
#plt.imshow(c512.get_subtraction_coarse_field()[0])
#print(np.mean(c512.get_subtraction_coarse_field()[0]))

In [ ]:
#plt.figure(figsize=(20,20))

#plt.imshow(original_field512 - c512.get_subtraction_coarse_field()[0])
#print(np.mean(original_field512 - c512.get_subtraction_coarse_field()[0]))

In [ ]:
algorithm_stages_plot(original_field512, removed_field512, added_field512, final_field512, c512.den_grid_size, 500, True)

In [ ]:
# Make 512 resolution algorithm stages plot with red box for Figure 1
# Zoom in on 140 Mpc
print(u)
algorithm_stages_plot(original_field512[0:2*u,0:2*u], removed_field512[0:2*u,0:2*u], added_field512[0:4*u*res,0:4*u*res], final_field512[0:4*u*res,0:4*u*res], c512.den_grid_size, 500, True)

In [ ]:
# Make the 256 vs 512 comparison 
# Zoom in on 140 Mpc
algorithm_stages_plot(original_field[0:u,0:u], removed_field[0:u,0:u], added_field[0:4*u*res,0:4*u*res], final_field[0:4*u*res,0:4*u*res], c.den_grid_size, 500, False)
algorithm_stages_plot(original_field512[0:2*u,0:2*u], removed_field512[0:2*u,0:2*u], added_field512[0:4*u*res,0:4*u*res], final_field512[0:4*u*res,0:4*u*res], c512.den_grid_size, 500, False)


In [ ]:
# Get data for the profile comparison of the fields (res 256)

c_STH2 = Configuration(SphericalTophatProfile(rvir_factor=2), resolution=resolution, provider=BolshoiProvider())
c_STH2.datestamp = date
STH2 = c_STH2.get_final_field()[0]

c_NFW = Configuration(NFWProfile(), resolution=resolution, provider=BolshoiProvider())
c_NFW.datestamp = date
NFW = c_NFW.get_final_field()[0]

c_FIRE = Configuration(FireProfile(), resolution=resolution, provider=BolshoiProvider())
c_FIRE.datestamp = date
FIRE = c_FIRE.get_final_field()[0]

c_P = Configuration(PrecipitationProfile(), resolution=resolution, provider=BolshoiProvider())
c_P.datestamp = date
PRE = c_P.get_final_field()[0]


In [ ]:
# Make bottom part of Figure 1. Comparies 4 CGM profiles. Zooms in a lot.
vmin = 0
vmax = 500
filename = "implot_add_remove_halos_256_all.pdf"

fig, axs = fields_comparison_plot([('NFW', NFW[0:p,0:p]), ('FIRE', FIRE[0:p,0:p]), ('Precipitation', PRE[0:p,0:p]), ('Tophat $2R_{vir}$', STH2[0:p,0:p])], vmin, vmax)
saveFig(filename, fig, bbox_inches='tight')
